# OpenAI Functions

Ok, let's first understand how [OpenAI](https://openai.com/) the company behind ChatGPT, allows for these function call implementations in its API.

OpenAI implemented a [function calling API](https://platform.openai.com/docs/guides/function-calling) which is a standard way to connect their models to outside tools like in the very simple example we did above.

According to their [official documentation](https://platform.openai.com/docs/guides/function-calling#:~:text=The%20basic%20sequence,to%20the%20user.) the sequence of steps for function calling is as follows:
1. Call the model with the user query and a set of functions defined in the functions parameter.
2. The model can choose to call one or more functions; if so, the content will be a stringified JSON object adhering to your custom schema (note: the model may hallucinate parameters).
3. Parse the string into JSON in your code, and call your function with the provided arguments if they exist.
4. Call the model again by appending the function response as a new message, and let the model summarize the results back to the user.

Below is an example taken from their official documentation:

In [ ]:
from openai import OpenAI
import json

client = OpenAI()

Let's look at how our previous model with those three simple functions: `create_directory()`, `create_file()`, and `list_files()` would be implemented using OpenAI's function calling approach:

In [ ]:
import json
import subprocess

def create_directory(directory_name):
    """Function that creates a directory given a directory name."""""
    subprocess.run(["mkdir", directory_name])
    return json.dumps({"directory_name": directory_name})


tool_create_directory = {
    "type": "function",
    "function": {
        "name": "create_directory",
        "description": "Create a directory given a directory name.",
        "parameters": {
            "type": "object",
            "properties": {
                "directory_name": {
                    "type": "string",
                    "description": "The name of the directory to create.",
                }
            },
            "required": ["directory_name"],
        },
    },
}

tools = [tool_create_directory]    

In [ ]:
import json

def run_terminal_task():
    messages = [{"role": "user", "content": "Create a folder called 'lucas-the-agent-master'."}]
    tools = [tool_create_directory]  
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-16k",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "create_directory": create_directory,
        }
        messages.append(response_message)
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                directory_name=function_args.get("directory_name"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-16k",
            messages=messages,
        )
        return second_response

output = run_terminal_task()
output

ChatCompletion(id='chatcmpl-8z55Zu8O8XgmDUc1jFlVOudLQmo4x', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I have created a folder called 'lucas-the-agent-master'.", role='assistant', function_call=None, tool_calls=None))], created=1709568261, model='gpt-3.5-turbo-16k-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=13, prompt_tokens=57, total_tokens=70))

In [ ]:
output.choices[0].message.content

"I have created a folder called 'lucas-the-agent-master'."

In [ ]:
!ls -d */

assets-resources/       images/                 persist_directory/
dev-notebooks/          lucas-the-agent-master/
docs/                   pages/


In [ ]:
from pydantic import BaseModel, Field
import openai
import os
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain.chat_models import ChatOpenAI

class InvoiceData(BaseModel):
    """Call this function to extract invoice data with specific output structure"""
    invoice_number: str = Field(..., description="The invoice number extracted from the PDF")
    vat_registration_number: str = Field(..., description="The VAT registration number extracted from the PDF")

fake_invoice_data = """
Invoice Summary
==============
Invoice No: INV-123456
Date: 2024-02-11
Billing To: ACME Corporation
Address: 123 Business Rd, Business City, BC1234

Product Description       Quantity    Unit Price    Total
-----------------------------------------------------------
Widget A                  10          $15.00        $150.00
Gadget B                  5           $20.00        $100.00

Subtotal:                                          $250.00
Tax (10%):                                          $25.00
Total:                                             $275.00

VAT Registration Number: VAT-7890123

Thank you for your business!

"""


invoice_extractor = convert_pydantic_to_openai_function(InvoiceData)


llm_chat = ChatOpenAI()

output_invoice = llm_chat.invoke(f"Invoice: {fake_invoice_data}. Extracted data:\n",functions=[invoice_extractor])
output_invoice.additional_kwargs["function_call"]